In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout



In [2]:
data = pd.read_excel('/content/Input data of transactions received from Online Marketplace partners.xlsx')

In [3]:
data = data.dropna()

In [4]:
data.head

<bound method NDFrame.head of     vendor_name                             vendor_id  consumer_id  MCC_code  \
0     bigbasket  7f20b039-9c49-4356-b571-32d96300b3b8         9812      1219   
2        amazon  ffe4098c-bc1b-489c-8e5b-cc8aeda9cea3         5124      1220   
3        amazon  ffe4098c-bc1b-489c-8e5b-cc8aeda9cea3         9190      1220   
4     bigbasket  7f20b039-9c49-4356-b571-32d96300b3b8         4123      1219   
5     bigbasket  7f20b039-9c49-4356-b571-32d96300b3b8         4123      1219   
..          ...                                   ...          ...       ...   
995     blinkit  2140b995-348b-4779-bb59-7741a21476b6         9122      1221   
996   bigbasket  7f20b039-9c49-4356-b571-32d96300b3b8         9133      1219   
997     blinkit  2140b995-348b-4779-bb59-7741a21476b6         9122      1221   
998      amazon  ffe4098c-bc1b-489c-8e5b-cc8aeda9cea3         3142      1220   
999      amazon  ffe4098c-bc1b-489c-8e5b-cc8aeda9cea3         5124      1220   

     Tran

In [ ]:
target = 'emission_footprint'
features = ['product_catetgory', 'from_city', 'to_city', 'Company']

label_encoders = {}
for feature in features:
    label_encoders[feature] = LabelEncoder()
    data[feature] = label_encoders[feature].fit_transform(data[feature])


X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(len(features),)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))


model.compile(loss='mean_squared_error', optimizer='adam')


model.fit(X_train, y_train, epochs=150, batch_size=50, verbose=1)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


new_data = pd.DataFrame({
    "product_catetgory": ["Stationary"],
    "from_city": ["Pune"],
    "to_city": ["Jaipur"],
    "Company": ["Camlin"]
})

for feature in features:
    new_data[feature] = label_encoders[feature].transform(new_data[feature])

prediction = model.predict(new_data)
print(f"Predicted emission footprint: {prediction[0][0]}")


In [ ]:
def predict_emission_footprint(model, label_encoders, features, new_data):
    predictions = []
    for data in new_data:
        new_data_df = pd.DataFrame({
            "product_catetgory": [data[0]],
            "from_city": [data[1]],
            "to_city": [data[2]],
            "Company": [data[3]]
        })

        for feature in features:
            new_data_df[feature] = label_encoders[feature].transform(new_data_df[feature])

        prediction = model.predict(new_data_df)
        predictions.append(prediction[0][0])

    lowest_prediction = min(predictions)
    return lowest_prediction

new_data = [
    ["Stationary", "Pune", "Jaipur", "Camlin"],
    ["Stationary", "Bhopal", "Lucknow", "Faber"],
    ["Sports", "Bangalore", "Chennai", "Nike"]
]

lowest_prediction = predict_emission_footprint(model, label_encoders, features, new_data)
print(f"Lowest predicted emission footprint: {lowest_prediction}")

1/1 [==============================] - 0s 21ms/step
Lowest predicted emission footprint: 20504.44921875


In [ ]:
import pickle
model_filename = "model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(model, file)
print(f"Trained model is saved as {model_filename}")


Trained model is saved as model.pkl


In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import pickle

from sklearn.linear_model import LinearRegression

data = pd.read_excel('/content/Input data of transactions received from Online Marketplace partners.xlsx') 
target = 'emission_footprint'
features = ['product_catetgory', 'from_city', 'to_city', 'Company']

label_encoders = {}
for feature in features:
    label_encoders[feature] = LabelEncoder()
    data[feature] = label_encoders[feature].fit_transform(data[feature])


X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

new_data = pd.DataFrame({
    "product_catetgory": ["Clothing"],
    "from_city": ["Bangalore"],
    "to_city": ["Chennai"],
    "Company": ["Zara"]
})

for feature in features:
    new_data[feature] = label_encoders[feature].transform(new_data[feature])

prediction = model.predict(new_data)
print(f"Predicted emission footprint: {prediction[0]}")

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)


Predicted emission footprint: 49153.87695156307
